<h1>Clean CSV Data and Add to SQL Database</h1>

<h4>Import Dependencies</h4>

In [1]:
import os
import csv
import pandas as pd

import sqlite3
import csv
from sqlalchemy import create_engine

from config import pgPassword

<h4>Create paths to CSV files</h4>

In [2]:
pathBeers = os.path.join("data", "beers.csv")
pathBreweries = os.path.join("data", "breweries.csv")
pathOpenBeer = os.path.join("data", "open-beer-database.csv")
pathRecipe = os.path.join("data", "recipeData.csv")

<h4>Load Beer file into DataFrame and clean</h4>

In [3]:
beersDF = pd.read_csv(pathBeers)
beersDF = beersDF.drop_duplicates(subset=['id'])
beersDF.sort_values(by=["id"]).head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
1583,1583,0.065,65.0,1,Dale's Pale Ale,American Pale Ale (APA),166,12.0
1582,1582,0.087,85.0,4,Gordon Ale (2009),American Double / Imperial IPA,166,12.0
1581,1581,0.080,35.0,5,Old Chub,Scottish Ale,166,12.0
1580,1580,0.099,100.0,6,GUBNA Imperial IPA,American Double / Imperial IPA,166,12.0
1579,1579,0.053,35.0,7,Mama's Little Yella Pils,Czech Pilsener,166,12.0


<h4>Load Breweries file into DataFrame and clean</h4>

In [4]:
breweriesDF = pd.read_csv(pathBreweries)
breweriesDF = breweriesDF.rename(columns={"Unnamed: 0": "id"})
breweriesDF = breweriesDF.drop_duplicates(subset=['id'])
breweriesDF.head(167)

,id,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA
...,...,...,...,...
162,162,Right Brain Brewery,Traverse City,MI
163,163,Sly Fox Brewing Company,Phoenixville,PA
164,164,Round Guys Brewing,Lansdale,PA
165,165,Great Crescent Brewery,Aurora,IN


<h4>Load OpenBeer file into DataFrame and clean</h4>

In [5]:
openBeerDF = pd.read_csv(pathOpenBeer)
targetColumns = ["Name", "id", "Alcohol_By_Vol", "IBU", "Style", "style_id", "Category","Brewer","brewery_id", "Address", "City", "State", "Country", "Coordinates"]
openBeerDF = openBeerDF[targetColumns]
openBeerDF = openBeerDF.drop_duplicates(subset=['id'])
openBeerDF.sort_values(by=["id"]).head()

,Name,id,Alcohol_By_Vol,IBU,Style,style_id,Category,Brewer,brewery_id,Address,City,State,Country,Coordinates
391,Hocus Pocus,1,5,0,Light American Wheat Ale or Lager,116,Other Style,Magic Hat,812,5 Bartlett Bay Road,South Burlington,Vermont,United States,"44.4284, -73.2131"
473,Grimbergen Blonde,2,7,0,na,-1,na,Brouwerij Alken-Maes,264,Rue Derbque 7,Jumet,Hainaut,Belgium,"50.4431, 4.4147"
2937,Widdershins Barleywine,3,9,0,na,-1,na,Left Hand Brewing Company,779,1265 Boston Avenue,Longmont,Colorado,United States,"40.1587, -105.113"
2938,Lucifer,4,9,0,na,-1,na,Brouwerij Liefmans,287,200 Aalststraat,Oudenaarde,Oost-Vlaanderen,Belgium,"50.8439, 3.617"
2082,Bitter,5,4,0,na,-1,na,Ridgeway Brewing,1056,6 Chapel Close,South Stoke,Oxford,United Kingdom,"51.5462, -1.1355"


<h4>Load Recipe file into DataFrame and clean</h4>

In [6]:
recipesDF = pd.read_csv(pathRecipe, encoding = "ISO-8859-1")
recipesDF = recipesDF[["BeerID", "Name", "Style", "StyleID","OG", "FG", "ABV","IBU", "Color", "Efficiency", "BrewMethod" ]]
recipesDF = recipesDF.drop_duplicates(subset=['BeerID'])
recipesDF.head()

,BeerID,Name,Style,StyleID,OG,FG,ABV,IBU,Color,Efficiency,BrewMethod
0,1,Vanilla Cream Ale,Cream Ale,45,1.055,1.013,5.48,17.65,4.83,70.0,All Grain
1,2,Southern Tier Pumking clone,Holiday/Winter Special Spiced Beer,85,1.083,1.021,8.16,60.65,15.64,70.0,All Grain
2,3,Zombie Dust Clone - EXTRACT,American IPA,7,1.063,1.018,5.91,59.25,8.98,70.0,extract
3,4,Zombie Dust Clone - ALL GRAIN,American IPA,7,1.061,1.017,5.80,54.48,8.50,70.0,All Grain
4,5,Bakke Brygg Belgisk Blonde 50 l,Belgian Blond Ale,20,1.060,1.010,6.48,17.84,4.57,72.0,All Grain


<h4>Create a connection to SQL database</h4>

In [7]:
pg_user = 'postgres'
pg_password = pgPassword
db_name = 'beersDB'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
engine.table_names()

[]

<h4>Add dataframes to SQL database</h4>

In [9]:
beersDF.to_sql(name = "beers", con=engine, if_exists='append', index=False)

In [10]:
breweriesDF.to_sql(name = "breweries", con=engine, if_exists='append', index=False)

In [11]:
openBeerDF.to_sql(name = "openBeer", con=engine, if_exists='append', index=False)

In [12]:
recipesDF.to_sql(name = "recipes", con=engine, if_exists='append', index=False)

In [13]:
engine.table_names()

['beers', 'breweries', 'openBeer', 'recipes']

In [15]:
pd.read_sql_query('select * from beers', con=engine).head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


<h4>Pull SQL table back into DataFrame</h4>

In [16]:
newBeerDF = pd.read_sql_table('beers', con=engine)
newBeerDF.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


<h4>Convert Dataframe to Dictionary</h4>

In [17]:
newBeerDict = newBeerDF.to_dict('records')
print(newBeerDict[0])

{'Unnamed: 0': 0, 'abv': 0.05, 'ibu': nan, 'id': 1436, 'name': 'Pub Beer', 'style': 'American Pale Lager', 'brewery_id': 408, 'ounces': 12.0}


In [18]:
beersAndBreweries = pd.merge(left = beersDF, right = breweriesDF, how="inner", left_on="brewery_id", right_on = "id")
beersAndBreweries = beersAndBreweries[["name_x", "style", "name_y", "abv", "ibu"]]
beersAndBreweries = beersAndBreweries.rename(columns = {"name_x": "beer",
                                                       "name_y": "brewery"})
beersAndBreweries.head()

breweryLocations = openBeerDF[["Brewer", "Address", "City", "State", "Country", "Coordinates"]]

joinedDF = pd.merge(left = breweryLocations, right = beersAndBreweries, how="inner", left_on="Brewer", right_on = "brewery")

noNulls = joinedDF.dropna().drop_duplicates()

noNulls.head()

,Brewer,Address,City,State,Country,Coordinates,beer,style,brewery,abv,ibu
15,Carolina Brewery,460 West Franklin Street,Chapel Hill,North Carolina,United States,"35.9103, -79.0632",Santa's Secret,Winter Warmer,Carolina Brewery,0.059,22.0
27,Breckenridge Brewery,471 Kalamath Street,Denver,Colorado,United States,"39.7236, -105.001",Agave Wheat,American Pale Wheat Ale,Breckenridge Brewery,0.042,9.0
28,Breckenridge Brewery,471 Kalamath Street,Denver,Colorado,United States,"39.7236, -105.001",SummerBright Ale,American Pale Wheat Ale,Breckenridge Brewery,0.045,15.0
29,Breckenridge Brewery,471 Kalamath Street,Denver,Colorado,United States,"39.7236, -105.001",Lucky U IPA,American IPA,Breckenridge Brewery,0.062,68.0
30,Breckenridge Brewery,471 Kalamath Street,Denver,Colorado,United States,"39.7236, -105.001",Avalanche Ale,American Amber / Red Ale,Breckenridge Brewery,0.054,19.0


In [19]:
for i, r in noNulls.iterrows():
    print(r["Brewer"])

Carolina Brewery
Breckenridge Brewery
Breckenridge Brewery
Breckenridge Brewery
Breckenridge Brewery
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Nebraska Brewing Company
Deschutes Brewery
Flying Dog Brewery
Flying Dog Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
Stevens Point Brewery
O'Fallon Brewery
O'Fallon Brewery
Capital Brewery
Capital Brewery
Capital Brewery
Capital Brewery
Capital Brewery
Boulder Beer Company
Boulder Beer Company
Boulder Beer Company
Boulevard Brewing Company
Boston Beer Comp